In [37]:
# Read packedges

import pandas as pd
import pandas.testing as tm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.discrete.discrete_model as dm

from patsy import dmatrices
import statsmodels.graphics.tsaplots as tsa


from scipy.fft import fft, ifft, fftfreq

import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import acf

import itertools
from itertools import combinations, chain

from scipy.stats import pearsonr

import re

import functions

from datetime import datetime

import pymannkendall as mk

import math

import trend_timeseries

from scipy.stats import friedmanchisquare

from sklearn.metrics import r2_score

from pmdarima.preprocessing import FourierFeaturizer
from pmdarima.datasets import load_wineind
from sklearn.linear_model import LinearRegression

from statsmodels.tsa.seasonal import seasonal_decompose

import early_warning_detection_functions


# Read data

In [38]:
df_aih = pd.read_parquet('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Results/data_manuscript_warning_aih_imed2.parquet')


In [39]:
df_aih = df_aih[(df_aih.year_week >= '2022-47') & (df_aih.year_week <= '2024-52')]

# Prepare data and identify trend from the series

In [40]:
lst_dfs = []

for region in df_aih['co_imed'].unique():
    # Select data for a specific municipality
    set_imed = df_aih[df_aih['co_imed'] == region]

    # Sort by the epidemiological week date column
    set_imed = set_imed.sort_values(by='year_week')

    # Calculate 4-week rolling means and rename columns
    set_imed = set_imed.assign(aih_4 = set_imed['n'].rolling(window=4).mean())
        
    # Calculate lagged values
    for lag in range(1, 4):
        set_imed = set_imed.assign(**{f'aih_4_lag_{lag}': set_imed.aih_4.shift(lag)})
            
        # Fill NaN values with 0
        set_imed = set_imed.fillna(0)

    # Append the DataFrame to the list
    lst_dfs.append(set_imed)

## Run negative binomial regressions to identify trend

In [41]:
# Function to handle Negative Binomial regression
def fit_negative_binomial(series_name):
    if dta[series_name].isnull().all() or np.all(dta[series_name] == 0):
        
        return {
                f'coef_negbi_{series_name}': 0,
                f'std_err_negbi_{series_name}': 0,
                f'z_negbi_{series_name}': 0,
                f'p_values_negbi_{series_name}': 1,
                f'IC_low_negbi_{series_name}': 0,
                f'IC_high_negbi_{series_name}': 0,
                f'trend_line_negbi_{series_name}': np.zeros(len(dta))
                }
    
    else:
        model = smf.glm(
                formula=f'{series_name} ~ time_trend',
                data=dta,
                family=sm.families.NegativeBinomial(alpha=1)
                ).fit()
            
        return {
                f'coef_negbi_{series_name}': model.params['time_trend'],
                f'std_err_negbi_{series_name}': model.bse['time_trend'],
                f'z_negbi_{series_name}': model.tvalues['time_trend'],
                f'p_values_negbi_{series_name}': model.pvalues['time_trend'],
                f'IC_low_negbi_{series_name}': model.conf_int().loc['time_trend', 0],
                f'IC_high_negbi_{series_name}': model.conf_int().loc['time_trend', 1],
                f'trend_line_negbi_{series_name}': model.predict()
                }


In [42]:
lst_results = []  # Initialize an empty list to store the processed DataFrames

for dta in lst_dfs:
    # Add a time trend column
    dta = dta.assign(time_trend=np.arange(len(dta)))

    # Fit Negative Binomial regression and update DataFrame
    negbi_serie = fit_negative_binomial('aih_4')
        

    dta = dta.assign(**negbi_serie)

    # Append the processed DataFrame to the list
    lst_results.append(dta)


In [43]:
df_aih2 =  pd.concat(lst_results, ignore_index=True)

In [44]:
print('Number of regions with significant trend')
df_aih2[df_aih2.p_values_negbi_aih_4 <= 0.05].co_imed.nunique()

Number of regions with significant trend


29

# Friedman test to Test Seasonality Significance

In [45]:
def friedman_test(x, freq=None):
    """
    Conducts a Friedman rank test for seasonality in a time series.
    
    Parameters:
    - x: pandas Series or numpy array, the time series data.
    - freq: int, the frequency of the time series.
    
    Returns:
    - dict: Contains test statistic, p-value, and additional information.
    """
    
    # Reshape into matrix form for Friedman test
    rows = len(x) // freq
    if rows < 2:
        raise ValueError("Not enough data points for the specified frequency.")
    
    x = x[:rows * freq]  # Truncate to fit matrix dimensions
    data_matrix = np.reshape(x, (rows, freq))
    
    # Perform Friedman test
    test_stat, p_value = friedmanchisquare(*data_matrix.T)
    
    # Output results
    return {
        "test_stat": test_stat,
        "p_value": p_value,
        "rows": rows,
        "columns": freq
    }


In [46]:
lst = []
for data in lst_results:

    # Keep 'data' as a DataFrame
    data = data.assign(dtrend_aih_negbi = data.aih_4 - data.trend_line_negbi_aih_4)
    
    # Convert 'dtrend_ivas_negbi' column to NumPy array for Friedman test
    data_array = data['dtrend_aih_negbi'].to_numpy()

    # Set period for decomposition
    p = len(data_array) // 2

    # Perform Friedman test
    res_test = friedman_test(data.dtrend_aih_negbi.to_numpy(), freq= p)

    # Ensure '*_4_sea' is not zero
    data['aih_4_sea'] = data['aih_4'].to_numpy() + 1e-5  

    # Perform seasonal decomposition
    result = seasonal_decompose(
        data.set_index('year_week')['aih_4_sea'], 
        model='multiplicative',  # Fixed spelling
        period=p, 
        extrapolate_trend='freq'
    )

    # Store results back 
    data = data.assign(p_value_aih_negbi_friedman   = res_test['p_value'],
                       test_stat_aih_negbi_friedman = res_test['test_stat'],
                       seas_decom_aih=result.seasonal.to_numpy()
                       )
    lst.append(data)

In [47]:
df_aih2 =  pd.concat(lst, ignore_index=True)

In [48]:
print('Number of imediate regions with significant sezonality')
df_aih2[df_aih2.p_value_aih_negbi_friedman <= 0.05].co_imed.nunique()

Number of imediate regions with significant sezonality


444

# Save data

In [16]:
df_aih2.to_parquet('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Results/data_manuscript_warning_aih_imed_for_regre.parquet')

